In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import yfinance as yf 


import torch
import sklearn.preprocessing import MinMaxScaler


import datetime
import cv2

In [ ]:
# company ticker symbol
ticker = 'NVDA'
df = yf.download(ticker)

In [ ]:
# show stock plot over the years
df.Close.plot(figure=(12,8))

In [ ]:
scaler = StandardScaler()
df['Close'] = scalar.fit_transform(df['Close'])


### Trading Environment

In [ ]:
class TradingEnv:
    def __init__(self, data, window_size = 30, initial_balance = 10000):
        self.data = data
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.reset()
    
    def reset(self):
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.shares_held = 0
        self.net_worth = self.initial_balance
        self.done = False
        return self.getStates()
    
    def getStates(self):
        state = self.data[self.current_step - self.window_size:self.current_step].values
        return np.concatenate([state.flatten(), [self.balance, self.shares_held]])
    
    def step(self, action):
        price = self.data.iloc[self.current_step]['close']
        prev_net = self.net_worth
        reward = 0

        if action == 1: # Buy
            if self.balance >= price:
                self.balance -= price
                self.shares_held += 1 
        elif action == 2: # sell
            if self.shares_held > 0:
                self.balance += price 
                self.shares_held -= 1
                reward = 1
    
    self.current_step += 1
    self.done = self.current_step >= len(self.df)
    self.net_worth = self.balance + self.shares_held * price
    reward = self.net_worth - prev_net

    return self.getStates(), reward, self.done

Network architecture using leaky ReLU

In [ ]:
N_features = 5 # OHLCV
window_size = 30 # 30-day trading window
N_states = N_features * window_size 

class TradingNet(torch.nn.Module):
    def __init__(self):
        super(TradingNet, self).__init__()
        self.fc1 = torch.nn.Linear(N_states,8192)
        self.fc2 = torch.nn.Linear(8192,1024)
        self.fc3 = torch.nn.Linear(1024,3)
        self.activ = torch.nn.LeakyReLU()
    
    def forward(self, x):
        x =self.activ(self.fc1(x))
        x = self.activ(self.fc2(x))
        return self.fc3(x)

### Replay Class

In [ ]:
'''Replay class will be constructed here'''

In [ ]:
'''Initial model settings will be here '''

### Get data

In [ ]:
def stock_data(ticker, start="2020-01-01", end="2023-12-31"):
    #load stock data while also cleaning columns of interest
    data = yf.download(ticker, start=start, end=end)[["Open","High","Low","Close","Volume"]].fillna(method='ffill')
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data

In [ ]:
def train():
    data = stock_data()
    env = TradingEnv(data,30)#30 is for days
    action_size = 3 # buy,sell, hold
    model = TradingNet() # neural network

    #network setup
    target_model = TradingNet()
    target_model.load_state_dict(model.state_dict())

    net = TradingNet().cuda()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
    criterion = torch.nn.MSELoss()

    # Training Loop
    for episode in range(EPISODES):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            if np.random.rand() <= EPSILON:
                action = random.randrange(action_size)
            else:
                with torch.no_grad():
                    action = torch.argmax(model(torch.FloatTensor(state))).item()
            
            next_state, reward, done = env.step(action)
            replay_buff.push(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            # Replay



